# Time Series Forecasting, Methods and Trends

**Ronny A. Meza A. and Alberto Fernández H**

# Introduction

This notebook is the complement of an article with the same name. In this notebook we are to share the data and methodology used for to reach the goals. The main objective will be to evaluate the evolution of the time series forecasting methods and the different topics to which it was applied over time. The main objective will be to evaluate the evolution of the time series forecasting methods and the different topics to which it was applied over time. In this way, we downloaded the references from Web of Sciences, with the following search criterias: Query: (TS=(time* AND series* AND forecasting)) AND LANGUAGE: (English) AND DOCUMENT TYPES: (Article OR Book Chapter OR Data Paper OR Database Review). The references with the all fields were downloaded in batches of 500 until until completing all the references of the search. For more information about getting bibliography metadata consult the following link: Bibliographyc Metadata.

The analysis was developed using Tethne Python package. Tethne is a Python software package for parsing and analyzing bibliographic metadata; This tool was developed by the Laubichler Lab and the Digital Innovation Group at Arizona State University as part of an initiative for digital and computational humanities (d+cH). It is important to take in consideration that Tethne is developed in Python 2.7. Python 3 is a distinct language from Python 2.7, and many packages have not yet made the leap to this new platform. A Python 3-compatible version is in the works.

In this project we use Tethne because, it provides tools for easily parsing and analyzing bibliographic data in Python. The primary emphasis is on working with data from the Web of Science database, and also, Tethne provides efficient methods for modeling and analyzing topics changing in time.

In [1]:
#Libraries
#%reset
%pylab inline
from tethne.readers import wos
from tethne.readers import merge
import matplotlib.pyplot as plt
from tethne import networks
from tethne.writers.graph import to_graphml
from tethne import bibliographic_coupling
from tethne.networks import authors
from tethne.utilities import _iterable
from tethne import GraphCollection
from tethne.networks import coauthors
import networkx as nx
from collections import defaultdict
import warnings
from tethne import networks
from pprint import pprint
from tethne import tokenize
from tethne.model.corpus import mallet
from tethne.networks import topics
from tethne.writers import graph
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import seaborn as sns
from tethne.analyze.corpus import burstness
from tethne.plot import plot_burstness
import warnings
warnings.filterwarnings("ignore")

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Populating the interactive namespace from numpy and matplotlib


In [ ]:
wos_corpus = wos.read('/home/ronny/Documentos/Artículos Doctorado/jupyter_codes/WOS')

## 2. Methodology
References from **Web of Sciences** with the following search parameters: *(TS=(time* AND series* AND forecasting)) AND LANGUAGE: (English) AND DOCUMENT TYPES: (Article OR Book Chapter OR Data Paper OR Database Review).* `The time windows is from 1900 to march, 15, 2019`. 

The Tethne' library called **read** can to load a list of Papers from a directory containing multiple data files; in our case, we have 31 records. Only it is necesary to provide the path to a directory containing several WoS field-tagged data files. The read function knows that your path is a directory and not a data file. 

In [ ]:
wos_corpus = wos.read('/home/ronny/Documentos/Artículos Doctorado/Article_1_py/WOS')

In [ ]:
ref_num = len(wos_corpus)
print 'Loaded %i Web of Sciences references' % ref_num

In [ ]:
#Creating graphics with Latex properties
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

from __future__ import unicode_literals
import matplotlib
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.unicode'] = True

In [ ]:
#Note that we can look how documents themselves are distributed using the distribution() method.

#plt.rcParams["figure.figsize"]= [6.4, 4]
plt.rcParams["figure.figsize"]= [16, 6]

plt.bar(*wos_corpus.distribution(), color='gray', edgecolor='black')    # <-- The action.

plt.xlabel(r'\textbf{YEAR}', fontsize=12)
plt.ylabel(r'\textbf{NUMBER OF DOCUMENTS}', fontsize=12)
plt.title(r'\textbf{TIME SERIES FORECASTING PUBLICATIONS PER YEAR}', fontsize=14, color='k')
plt.grid(True)
#plt.savefig('figure_1')
plt.show()

## 2.1 WoS Abstracts

The abstract_to_features method converts all of the available abstracts in our Corpus to a unigram featureset. It takes no arguments. The abstracts will be diced up into their constituent words, punctuation and capitalization is removed, and a featureset called abstractTerms is generated. By default, abstract_to_features will apply the NLTK stoplist and Porter stemmer.

In [ ]:
wos_corpus.index_feature('abstract', tokenize=tokenize, structured=True)

In [ ]:
wos_corpus.features.keys()

In [ ]:
print 'There are {0} features in the abstract featureset.'.format(len(wos_corpus.features['abstract'].index))

In [ ]:
print 'Only {0} of {1} papers have abstracts'.format(len(wos_corpus.features['abstract'].features), len(wos_corpus.papers))

In [ ]:
print 'This corpus contains the following features: \n\t%s' % '\n\t'.join(wos_corpus.features.keys())

## 3. Analyzing research topics in time series forecasting, each 10 years 
**Dividing the Population**

In [ ]:
corpus_1940 = wos_corpus.subcorpus(('date', range(1940, 1990)))
corpus_1990 = wos_corpus.subcorpus(('date', range(1991, 2000)))
corpus_2000 = wos_corpus.subcorpus(('date', range(2001, 2010)))
corpus_2010 = wos_corpus.subcorpus(('date', range(2011, 2020)))

In [ ]:
#Function for determinant the corpus length
def len_corpus(corpus_data):
    ref_num1 = len(corpus_data)
    print 'The subcorpus has %i references' % ref_num1

In [ ]:
len_corpus(corpus_1940)
len_corpus(corpus_1990)
len_corpus(corpus_2000)
len_corpus(corpus_2010)

In [ ]:
#Subcorpus 1
plt.bar(*corpus_1940.distribution(), color='gray', edgecolor='black')    # <-- The action.
plt.xlabel(r'\textbf{YEAR}', fontsize=11)
plt.ylabel(r'\textbf{NUMBER OF DOCUMENTS}', fontsize=11)
plt.title(r'\textbf{PUBLICATIONS (1940 - 1990)}', fontsize = 14, color='k')
plt.grid(True)

In [ ]:
#Subcorpus 2
plt.bar(*corpus_1990.distribution(), color='gray', edgecolor='black')    # <-- The action.
plt.xlabel(r'\textbf{YEAR}', fontsize=11)
plt.ylabel(r'\textbf{NUMBER OF DOCUMENTS}', fontsize=11)
plt.title(r'\textbf{PUBLICATIONS (1990 - 2000)}', fontsize = 14, color='k')
plt.grid(True)

In [ ]:
#Subcorpus 3
plt.bar(*corpus_2000.distribution(), color='gray', edgecolor='black')    # <-- The action.
plt.xlabel(r'\textbf{YEAR}', fontsize=11)
plt.ylabel(r'\textbf{NUMBER OF DOCUMENTS}', fontsize=11)
plt.title(r'\textbf{PUBLICATIONS (2000 - 2010)}', fontsize = 14, color='k')
plt.grid(True)

In [ ]:
#Subcorpus 4
plt.bar(*corpus_2010.distribution(), color='gray', edgecolor='black')    # <-- The action.
plt.xlabel(r'\textbf{YEAR}', fontsize=11)
plt.ylabel(r'\textbf{NUMBER OF DOCUMENTS}', fontsize=11)
plt.title(r'\textbf{PUBLICATIONS (2010 - 2019)}', fontsize = 14, color='k')
plt.grid(True)
#plt.savefig('subcorpus')

## Subcorpus Keywords Analysis (1940-1990)

In [ ]:
#wos_corpus.index('authorKeywords')

corpus_1940.index_feature('authorKeywords')
corpus_1940.features.keys()

In [ ]:
store = list()
featureset = corpus_1940.features['authorKeywords']
for k, count in featureset.documentCounts.items():
    store.append((featureset.index[k], count))

Keywords_1940 = sorted(store, key=lambda x: x[1], reverse=True)

print(Keywords_1940[:200])

In [ ]:
#Function for search element inside the tuple and show in reverse order
def search_tuple(data, element):
    index = next((i for i,v in enumerate(data) if v[0] == element),-1)
    return(index)

In [ ]:
#Searching Time Series Forecasting Method' indexes
winter = search_tuple(Keywords_1940, 'HOLT-WINTERS')
winter2 = search_tuple(Keywords_1940, 'HOLT\u2010WINTERS')

box1 = search_tuple(Keywords_1940, 'BOX-JENKINS')
box2 = search_tuple(Keywords_1940, 'BOX\u2010JENKINS')
box3 = search_tuple(Keywords_1940, 'BOX-JENKINS MODELS')
box4 = search_tuple(Keywords_1940, 'BOX-JENKINS ANALYSIS')

arma1 = search_tuple(Keywords_1940, 'ARMA')
arma2 = search_tuple(Keywords_1940, 'IARMA')
arma3 = search_tuple(Keywords_1940, 'ARARMA MODEL')

arima1 = search_tuple(Keywords_1940, 'ARIMA MODELS')
arima2 = search_tuple(Keywords_1940, 'ARIMA')
arima3 = search_tuple(Keywords_1940, 'ARIMA MODEL')
arima4 = search_tuple(Keywords_1940, 'MULTIVARIATE ARIMA MODELS')

smooth = search_tuple(Keywords_1940, 'EXPONENTIAL SMOOTHING')

stochast = search_tuple(Keywords_1940, 'STOCHASTIC')

bayes = search_tuple(Keywords_1940, 'BAYESIAN')

regress1 = search_tuple(Keywords_1940, 'REGRESSION')
regress2 = search_tuple(Keywords_1940, 'REGRESSION ANALYSIS')
regress3 = search_tuple(Keywords_1940, 'ROBUST REGRESSION')
regress4 = search_tuple(Keywords_1940, 'REGRESSION, DYNAMIC REGRESSION')
regress5 = search_tuple(Keywords_1940, 'TIME-SERIES REGRESSION')

kalman = search_tuple(Keywords_1940, 'KALMAN FILTER')

In [ ]:
#Unification list for different forecasting methods

#HOLT-WINTERS
winter_id = Keywords_1940[winter][0]
winter_ref = Keywords_1940[winter][1] + Keywords_1940[winter2][1]
winter_ind = (winter_id, winter_ref)

#BOX-JENKINS
box_id = Keywords_1940[box1][0]
box_ref = Keywords_1940[box1][1] + Keywords_1940[box2][1] + Keywords_1940[box3][1] + Keywords_1940[box4][1]
box_ind = (box_id, box_ref)

#ARMA
arma_id = Keywords_1940[arma1][0]
arma_ref = Keywords_1940[arma1][1] + Keywords_1940[arma2][1] + Keywords_1940[arma3][1]
arma_ind = (arma_id, arma_ref)

#ARIMA
arima_id = Keywords_1940[arima2][0]
arima_ref = Keywords_1940[arima1][1] + Keywords_1940[arima2][1] + Keywords_1940[arima3][1] + Keywords_1940[arima4][1]
arima_ind = (arima_id, arima_ref)

#REGRESSION
regress_id = Keywords_1940[regress1][0]
regress_ref = Keywords_1940[regress1][1] + Keywords_1940[regress2][1] + Keywords_1940[regress3][1] + Keywords_1940[regress4][1] + Keywords_1940[regress5][1]
regress_ind = (regress_id, regress_ref)


In [ ]:
#Bar graph for methods

methods = [Keywords_1940[stochast], Keywords_1940[bayes], Keywords_1940[smooth],
           Keywords_1940[kalman], winter_ind, box_ind, arma_ind, arima_ind, regress_ind]

# sort in-place from highest to lowest
methods.sort(key=lambda x: x[1], reverse=False) 

# save the names and their respective scores separately
# reverse the tuples to go from most frequent to least frequent 
topic = zip(*methods)[0]
score = zip(*methods)[1]

x_pos = np.arange(len(topic)) 

plt.barh(x_pos, score, align='center', color='gray', edgecolor='black')
plt.yticks(x_pos, topic)
plt.xlabel(r'\textbf{NUMBER OF DOCUMENTS WHERE THE METHOD WAS MENTIONED}')
plt.ylabel(r'\textbf{METHODS}')
plt.title(r'\textbf{SOME FORECASTING METHODS THAT HAVE BEEN USED BETWEEN 1940-1990}', fontsize=14, color='k')

plt.grid(True)
#plt.savefig('figure_2')
plt.show()

In [ ]:
#Searching Research Topics index
#ECONOMIC
economic1 = search_tuple(Keywords_1940, 'ECONOMIC')
economic2 = search_tuple(Keywords_1940, 'ECONOMETRIC MODELS')
economic3 = search_tuple(Keywords_1940, 'ECONOMETRIC FORECASTING')
economic4 = search_tuple(Keywords_1940, 'ECONOMETRIC MODEL')
economic5 = search_tuple(Keywords_1940, 'ECONOMIC TIME SERIES')

#SALES
sales1 = search_tuple(Keywords_1940, 'SALES')
sales2 = search_tuple(Keywords_1940, 'SALES FORECASTING')

#ENERGY
energy1 = search_tuple(Keywords_1940, 'ENERGY')
energy2 = search_tuple(Keywords_1940, 'POWER')

demand = search_tuple(Keywords_1940, 'DEMAND')

weather = search_tuple(Keywords_1940, 'WEATHER')

market = search_tuple(Keywords_1940, 'MARKET')

finance = search_tuple(Keywords_1940, 'FINANCIAL')

In [ ]:
#Unification list for different forecasting methods

#ECONOMIC
economic_id = Keywords_1940[economic1][0]
economic_ref = Keywords_1940[economic1][1] + Keywords_1940[economic2][1] + Keywords_1940[economic3][1] + Keywords_1940[economic4][1] + Keywords_1940[economic5][1]
economic_ind = (economic_id, economic_ref)

#SALES
sales_id = Keywords_1940[sales1][0]
sales_ref = Keywords_1940[sales1][1] + Keywords_1940[sales2][1]
sales_ind = (sales_id, sales_ref)

#ENERGY
energy_id = Keywords_1940[energy1][0]
energy_ref = Keywords_1940[energy1][1] + Keywords_1940[energy2][1]
energy_ind = (energy_id, energy_ref)

In [ ]:
#Bar graph for topics

techniques = [Keywords_1940[demand], Keywords_1940[weather], Keywords_1940[market], Keywords_1940[finance],
           economic_ind, sales_ind, energy_ind]

# sort in-place from highest to lowest
techniques.sort(key=lambda x: x[1], reverse=False) 

# save the names and their respective scores separately
# reverse the tuples to go from most frequent to least frequent 
technique = zip(*techniques)[0]
score = zip(*techniques)[1]
#print(people)
x_pos = np.arange(len(technique)) 
   
plt.barh(x_pos, score, align='center', color='gray', edgecolor='black')
plt.yticks(x_pos, technique)
plt.xlabel(r'\textbf{NUMBER OF DOCUMENTS WHERE THE TOPIC WAS MENTIONED}')
plt.ylabel(r'\textbf{TECHNIQUES USED}')
plt.title(r'\textbf{SOME TOPICS THAT HAVE BEEN STUDIED BETWEEN 1940-1990}', fontsize=14, color='k')

plt.grid(True)
#plt.savefig('figure_3')
plt.show()


## Subcorpus Keywords Analysis (1991-2000)

In [ ]:
#wos_corpus.index('authorKeywords')

corpus_1990.index_feature('authorKeywords')
corpus_1990.features.keys()

In [ ]:
store = list()
featureset = corpus_1990.features['authorKeywords']
for k, count in featureset.documentCounts.items():
    store.append((featureset.index[k], count))

Keywords_1990 = sorted(store, key=lambda x: x[1], reverse=True)

print(Keywords_1990[:200])

In [ ]:
#Searching Time Series Forecasting Methods

#REGRESSION
regress1 = search_tuple(Keywords_1990, 'REGRESSION')
regress2 = search_tuple(Keywords_1990, 'TIME-SERIES REGRESSION')
regress3 = search_tuple(Keywords_1990, 'LINEAR-REGRESSION')
regress4 = search_tuple(Keywords_1990, 'INTERPOLATION')
regress5 = search_tuple(Keywords_1990, 'EXTRAPOLATION')

#AUTOREGRESSIVE
arima1 = search_tuple(Keywords_1990, 'AUTOREGRESSIVE TIME-SERIES')
arima2 = search_tuple(Keywords_1990, 'ARIMA')

#HETEROSCEDASTICITY
hetero1 = search_tuple(Keywords_1990, 'HETEROSKEDASTICITY')
hetero2 = search_tuple(Keywords_1990, 'CONDITIONAL HETEROSKEDASTICITY')
hetero3 = search_tuple(Keywords_1990, 'HETEROSCEDASTICITY')

#NEURAL NETWORKS
neural1 = search_tuple(Keywords_1990, 'NEURAL NETWORKS')
neural2 = search_tuple(Keywords_1990, 'MULTILAYER FEEDFORWARD NETWORKS')

lyapunov = search_tuple(Keywords_1990, 'LYAPUNOV EXPONENTS')

simu = search_tuple(Keywords_1990, 'SIMULATION')

In [ ]:
#Unification list for different forecasting methods

#REGRESSION
regress_id = Keywords_1990[regress1][0]
regress_ref = Keywords_1990[regress1][1] + Keywords_1990[regress2][1] + Keywords_1990[regress3][1] + Keywords_1990[regress4][1] + Keywords_1990[regress5][1]
regress_ind = (regress_id, regress_ref)


#AUTOREGRESSIVE
arima_id = Keywords_1990[arima2][0]
arima_ref = Keywords_1990[arima1][1] + Keywords_1990[arima2][1]
arima_ind = (arima_id, arima_ref)

#HETEROSCEDASTICITY
hetero_id = Keywords_1990[hetero1][0]
hetero_ref = Keywords_1990[hetero1][1] + Keywords_1990[hetero2][1] + Keywords_1990[hetero3][1]
hetero_ind = (hetero_id, hetero_ref)

#NEURAL NETWORKS
neural_id = Keywords_1990[neural1][0]
neural_ref = Keywords_1990[neural1][1] + Keywords_1990[neural2][1]
neural_ind = (neural_id, neural_ref)

In [ ]:
#Bar graph for topics
topics = [Keywords_1990[lyapunov], Keywords_1990[simu], regress_ind, arima_ind, hetero_ind, neural_ind]

# sort in-place from highest to lowest
topics.sort(key=lambda x: x[1], reverse=False) 

# reverse the tuples to go from most frequent to least frequent 
topic = zip(*topics)[0]
score = zip(*topics)[1]

x_pos = np.arange(len(topics)) 
  
plt.barh(x_pos, score, align='center', color='gray', edgecolor='black')
plt.yticks(x_pos, topic)
plt.xlabel(r'\textbf{NUMBER OF DOCUMENTS WHERE THE TECHNIQUE WAS MENTIONED}')
plt.ylabel(r'\textbf{METHODS}')
plt.title(r'\textbf{SOME TOPICS THAT HAVE BEEN STUDIED BETWEEN 1990-2000}', fontsize=14, color='k')

plt.grid(True)
#plt.savefig('figure_3')
plt.show()

In [ ]:
#Searching Time Series Forecasting Topics

#DEMAND
demand = search_tuple(Keywords_1990, 'DEMAND')

#RETURNS
returns1 = search_tuple(Keywords_1990, 'STOCK RETURNS')
returns2 = search_tuple(Keywords_1990, 'RETURNS')

#MARKET
market1 = search_tuple(Keywords_1990, 'MARKET')
market2 = search_tuple(Keywords_1990, 'MARKETS')

#ECONOMIC
economic1 = search_tuple(Keywords_1990, 'ECONOMIC TIME-SERIES')
economic2 = search_tuple(Keywords_1990, 'MACROECONOMIC TIME-SERIES')
economic3 = search_tuple(Keywords_1990, 'ECONOMIC FORECASTS')
economic4 = search_tuple(Keywords_1990, 'ECONOMY')
economic5 = search_tuple(Keywords_1990, 'INFLATION')
economic6 = search_tuple(Keywords_1990, 'INTEREST-RATES')
economic7 = search_tuple(Keywords_1990, 'RATES')

#WEATHER
weather1 = search_tuple(Keywords_1990, 'PRECIPITATION')
weather2 = search_tuple(Keywords_1990, 'CLIMATE')
weather3 = search_tuple(Keywords_1990, 'SEA-SURFACE TEMPERATURE')
weather4 = search_tuple(Keywords_1990, 'TEMPERATURES')
weather5 = search_tuple(Keywords_1990, 'WEATHER')
weather6 = search_tuple(Keywords_1990, 'WEATHER PREDICTION')
weather7 = search_tuple(Keywords_1990, 'SURFACE-TEMPERATURE')
weather8 = search_tuple(Keywords_1990, 'TEMPERATURE')
weather9 = search_tuple(Keywords_1990, 'RAINFALL')

#FINANCES
finances1 = search_tuple(Keywords_1990, 'VOLATILITY')
finances2 = search_tuple(Keywords_1990, 'STOCHASTIC VOLATILITY')                                               

In [ ]:
#Unification list for different forecasting topics

#RETURNS
returns_id = Keywords_1990[returns1][0]
returns_ref = Keywords_1990[returns1][1] + Keywords_1990[returns2][1]
returns_ind = (returns_id, returns_ref)

#MARKET
market_id = Keywords_1990[market2][0]
market_ref = Keywords_1990[market1][1] + Keywords_1990[market2][1]
market_ind = (market_id, market_ref)

#ECONOMIC
economic_id = Keywords_1990[economic4][0]
economic_ref = Keywords_1990[economic1][1] + Keywords_1990[economic2][1] + Keywords_1990[economic3][1] + Keywords_1990[economic4][1] + Keywords_1990[economic5][1] + Keywords_1990[economic6][1] + Keywords_1990[economic7][1]
economic_ind = (economic_id, economic_ref)

#WEATHER
weather_id = Keywords_1990[weather5][0]
weather_ref = Keywords_1990[weather1][1] + Keywords_1990[weather2][1] + Keywords_1990[weather3][1] + Keywords_1990[weather4][1] + Keywords_1990[weather5][1] + Keywords_1990[weather6][1] + Keywords_1990[weather7][1] + Keywords_1990[weather8][1] + + Keywords_1990[weather9][1]
weather_ind = (weather_id, weather_ref)

finances_id = Keywords_1990[finances1][0]
finances_ref = Keywords_1990[finances1][1] + Keywords_1990[finances2][1]
finances_ind = (finances_id, finances_ref)


In [ ]:
#Bar graph for topics

techniques = [Keywords_1990[demand], returns_ind, market_ind, economic_ind, weather_ind, finances_ind]

# sort in-place from highest to lowest
techniques.sort(key=lambda x: x[1], reverse=False) 

# save the names and their respective scores separately
# reverse the tuples to go from most frequent to least frequent 
technique = zip(*techniques)[0]
score = zip(*techniques)[1]

x_pos = np.arange(len(technique)) 
   
plt.barh(x_pos, score, align='center', color='gray', edgecolor='black')
plt.yticks(x_pos, technique)
plt.xlabel(r'\textbf{NUMBER OF DOCUMENTS WHERE THE TOPIC WAS MENTIONED}')
plt.ylabel(r'\textbf{TECHNIQUES USED}')
plt.title(r'\textbf{SOME TOPICS THAT HAVE BEEN STUDIED BETWEEN 1991-2000}', fontsize=14, color='k')

plt.grid(True)
#plt.savefig('figure_3')
plt.show()

In [ ]:
winter = search_tuple(Keywords_1990, 'HOLT-WINTERS')
winter = search_tuple(Keywords_1990, 'HOLT-WINTERS')
winter = search_tuple(Keywords_1990, 'HOLT-WINTERS')
winter = search_tuple(Keywords_1990, 'HOLT-WINTERS')
winter = search_tuple(Keywords_1990, 'HOLT-WINTERS')